# 0.0. Imports

In [ ]:
#Import Libraries
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from ydata_profiling import ProfileReport

from matplotlib import pyplot as plt

from xgboost import XGBClassifier

from sklearn import svm
from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp
from sklearn import linear_model as lm
from sklearn import ensemble as en

# 0.1. Functions

In [ ]:
#Rename dataframe columns
def rename_cols(df):
    cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
                'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
                'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
                'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 
                'Reserva_Cancelada', 'latitude', 'longitude']
    
    snakecase = lambda x: inflection.underscore(x)
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

def rename_cols_x(df):
    cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
                'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
                'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
                'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 'latitude', 'longitude']
    
    snakecase = lambda x: inflection.underscore(x)
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

#Suiting data to algorithm application
def feature_eng(df):
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))
    
    #Suiting data to algorithm application
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))
    
    #Suiting data to algorithm application
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))
    
    #Adjusting data types
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].astype('int64')
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].astype('int64')
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].astype('int64')
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].astype('int64')
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].astype('int64')
    
    return df

#Optimizing values for algorithm application
def feature_opt(df):
    #Make a frequency dictionary of each category
    freq_dict = df['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['tipo_do_quarto_reservado_encoded'] = df['tipo_do_quarto_reservado'].map(freq_dict)
    
    #Make a frequency dictionary of each category
    freq_dict = df['nacionalidade'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['nacionalidade'] = df['nacionalidade'].map(freq_dict)
    
    #Dummy variables
    cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']  
    df_dummy = pd.get_dummies(df[cols_dummy])  
    df = pd.concat([df, df_dummy], axis=1)
    
    #Adjusting values for algorithm application
    df['latitude'] = df['latitude'].str.replace('.', '')
    df['longitude'] = df['longitude'].str.replace('.', '')
    df['latitude'] = df['latitude'].astype('float64')
    df['longitude'] = df['longitude'].astype('float64')
    
    #Apply radians to data for optimize
    df['latitude'] = df['latitude'].apply(lambda x: np.radians(x/1000000))
    df['longitude'] = df['longitude'].apply(lambda x: np.radians(x/1000000))
    
    return df

def feature_opt_x(df):
    #Make a frequency dictionary of each category
    freq_dict = df['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['tipo_do_quarto_reservado_encoded'] = df['tipo_do_quarto_reservado'].map(freq_dict)
    
    #Make a frequency dictionary of each category
    freq_dict = df['nacionalidade'].value_counts(normalize=True).to_dict()
    
    #Apply encoder to dataframe
    df['nacionalidade'] = df['nacionalidade'].map(freq_dict)
    
    #Dummy variables
    cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']  
    df_dummy = pd.get_dummies(df[cols_dummy])  
    df = pd.concat([df, df_dummy], axis=1)
    
    #Adjusting values for algorithm application
    df['latitude'] = df['latitude'].str.replace('.', '')
    df['longitude'] = df['longitude'].str.replace('.', '')
    df['latitude'] = df['latitude'].astype('float64')
    df['longitude'] = df['longitude'].astype('float64')
    
    #Apply radians to data for optimize
    df['latitude'] = df['latitude'].fillna(40463667)
    df['longitude'] = df['longitude'].fillna(-374922)
    df['latitude'] = df['latitude'].apply(lambda x: np.radians(x/1000000))
    df['longitude'] = df['longitude'].apply(lambda x: np.radians(x/1000000))
    
    return df


#Adjusting dataframe types
def types_adjust(df):
    df['numero_de_hospedes'] = df['numero_de_hospedes'].astype('int64')
    df['classificacao_do_hotel'] = df['classificacao_do_hotel'].astype('int64')
    df['ja_se_hospedou_anterioremente'] = df['ja_se_hospedou_anterioremente'].astype('int64')
    df['reserva_feita_por_agencia_de_turismo'] = df['reserva_feita_por_agencia_de_turismo'].astype('int64')
    df['reserva_feita_por_empresa'] = df['reserva_feita_por_empresa'].astype('int64')
    df['reserva_com_estacionamento'] = df['reserva_com_estacionamento'].astype('int64')
    
    return df

# 1.0. Data Description

## 1.1. Loading Data

In [ ]:
#Import train data and country data (for merge)
df_train = pd.read_csv('train.csv')
df_country = pd.read_csv('country_base.csv', delimiter=';')

#Merge train and country data
df1 = pd.merge(df_train, df_country, on='Nacionalidade', how='left')

#Drop dataframe column
df1 = df1.drop('country', axis=1)

In [ ]:
#Import test data and country data (for merge)
df_test = pd.read_csv('test.csv')
df_country = pd.read_csv('country_base.csv', delimiter=';')

#Merge train and country data
df1_x = pd.merge(df_test, df_country, on='Nacionalidade', how='left')

#Drop dataframe column
df1_x = df1_x.drop('country', axis=1)

## 1.2. Rename Columns

In [ ]:
#Applying Rename Columns Function
df1 = rename_cols(df1)
df1_x = rename_cols_x(df1_x)

## 1.3. Data Dimensions

In [ ]:
#Counting number of rows and columns of the dataframe
print('Train Number of rows: {}'.format(df1.shape[0]))
print('Train Number of cols: {}'.format(df1.shape[1]))
print('\n')
print('Test Number of rows: {}'.format(df1_x.shape[0]))
print('Test Number of cols: {}'.format(df1_x.shape[1]))

## 1.4. Check NA

In [ ]:
#Check of NA's values os de dataframe
df1.isna().sum()

In [ ]:
df1_x.isna().sum()

### 1.4.1. Drop / Replace NA's

In [ ]:
#Drop NA's from train dataframe
df1 = df1.dropna()

#Replace NA's from test dataframe
df1_x['numero_de_hospedes'] = df1_x['numero_de_hospedes'].fillna(2)
df1_x['nacionalidade'] = df1_x['nacionalidade'].fillna('Spain')
df1_x['latitude'] = df1_x['latitude'].fillna(40463667)
df1_x['longitude'] = df1_x['longitude'].fillna(-374922)

## 1.5. Data Dtypes

In [ ]:
#Checking train data dtypes
df1.dtypes

In [ ]:
#Checking test data dtypes
df1_x.dtypes

## 1.6. Data Balancing

In [ ]:
#Checking test data balance
df1['reserva_cancelada'].value_counts(normalize=True)

## 1.7. Pandas Profiling

In [ ]:
#Report for External Data Analysis
prof = ProfileReport(df1)
prof.to_file(output_file='output.html')
# regime_de_alimentacao -> desbalanceado
# nacionalidade -> desbalanceado
# forma_de_reserva -> desbalanceado
# ja_se_hospedou_anterioremente -> desbalanceado
# tipo_do_quarto_reservado -> desbalanceado
# reserva_feita_por_agencia_de_turismo -> desbalanceado
# reserva_com_estacionamento -> desbalanceado

# 2.0. Feature Engineering

In [ ]:
#Dataframe copy for data adjustment
df2 = df1.copy()
df2_x = df1_x.copy()

In [ ]:
#Applying Feature Engineering Function
df2 = feature_eng(df2)
df2_x = feature_eng(df2_x)

In [ ]:
#Checking data adjustments
df2

In [ ]:
#Checking data types
df2.dtypes

In [ ]:
df2_x

In [ ]:
df2_x.dtypes

# 3.0. Data Filtering

In [ ]:
#This section was not used for this project

# 4.0. Data Preparation 

In [ ]:
#Dataframe copy for data preparation
df3 = df2.copy()
df3_x = df2_x.copy()

In [ ]:
#Applying Feature Optimizer Function
df3 = feature_opt(df3)

df3_x = feature_opt_x(df3_x)

In [ ]:
#Checking train data types
df3.dtypes

In [ ]:
#Checking test data types
df3_x.dtypes

In [ ]:
#Checking train data optimized
df3[['tipo_do_quarto_reservado', 'tipo_do_quarto_reservado_encoded', 'nacionalidade', 'latitude', 'longitude']]

In [ ]:
#Checking test data optimized
df3_x[['tipo_do_quarto_reservado', 'tipo_do_quarto_reservado_encoded', 'nacionalidade', 'latitude', 'longitude']]

# 5.0. Feature Selection 

In [ ]:
#Dataframe copy for feature selection
df4 = df3.copy()
df4_x = df3_x.copy()

In [ ]:
#Applying data types adjustment functions
df4 = types_adjust(df4)
df4_x = types_adjust(df4_x)

In [ ]:
#Checking train data
df4.dtypes

In [ ]:
#Checking test data
df4_x.dtypes

In [ ]:
#Drop columns from dataframe
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)
df5_x = df4_x.drop(cols_drop, axis=1)

In [ ]:
#Checking train data types for Machine Learning Application
df5.dtypes

In [ ]:
#Checking test data types for Machine Learning Application
df5_x.dtypes

In [ ]:
#Split train dataset into features and target
#X = df5.drop(['id', 'reserva_cancelada'], axis=1)
X = df5.drop(['reserva_cancelada'], axis=1)
y = df5['reserva_cancelada']

In [ ]:
#Split train dataset into train and test
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [ ]:
#Model Definition
xgb_model = XGBClassifier(n_estimators=1000, max_depth=20, min_child_weight=1)

#Model Training
xgb_model.fit(X_train, y_train.values.ravel())

#Model Prediction
yhat_xgb = xgb_model.predict(X_val)

In [ ]:
#Export the model pkl for test data application
#pickle.dump(xgb_model, open('xgb_model.pkl', 'wb' ))

### 6.1.1. Performance

In [ ]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_xgb)
recall = mt.recall_score(y_val, yhat_xgb)
f1_score = mt.f1_score(y_val, yhat_xgb, average='macro')

In [ ]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

## 6.2. Gradient Boosting

In [ ]:
#Model Definition
gdr_model = en.GradientBoostingClassifier(n_estimators=500, random_state=42, max_depth=15)

#Model Training
gdr_model.fit(X_train, y_train)

#Model Prediction
yhat_gb = gdr_model.predict(X_val)

In [ ]:
#Export the model pkl for test data application
#pickle.dump(gdr_model, open('gdr_model.pkl', 'wb' ))

### 6.2.1. Performance

In [ ]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_gb)
recall = mt.recall_score(y_val, yhat_gb)
f1_score = mt.f1_score(y_val, yhat_gb, average='macro')

In [ ]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

## 6.3. Random Forest

In [ ]:
#Model Definition
rf_model = en.RandomForestClassifier(n_estimators=1000,
                                     criterion='gini', 
                                     random_state=42, 
                                     n_jobs=-1, 
                                     max_depth=25)

#Model Training
rf_model.fit(X_train, y_train)

#Model Prediction
yhat_rf = rf_model.predict(X_val)

In [ ]:
#Export the model pkl for test data application
#pickle.dump(rf_model, open('rf_model.pkl', 'wb' ))

### 6.3.1. Performance

In [ ]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_rf)
recall = mt.recall_score(y_val, yhat_rf)
f1_score = mt.f1_score(y_val, yhat_rf, average='macro')

In [ ]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

# 7.0. Ensemble

In [ ]:
#Merge data to compare
ensemble = pd.DataFrame([yhat_xgb, yhat_gb, yhat_rf]).T
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']
moda = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda

# 8.0. General Performance

In [ ]:
#Collect values for test performance
yhat_en = ensemble['pred']

In [ ]:
#Show metrics of performance
precision = mt.precision_score(y_val, yhat_en)
recall = mt.recall_score(y_val, yhat_en)
f1_score = mt.f1_score(y_val, yhat_en, average='macro')

In [ ]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

# 9.0. Model Application to Test Data

In [ ]:
#XGBoost application to test data
yhat_xbg_x = xgb_model.predict(df5_x)

#Gradient Boosting application to test data
yhat_gb_x = gdr_model.predict(df5_x)

#Random Forest application to test data
yhat_rf_x = rf_model.predict(df5_x)

# 10.0. Export CSV

In [ ]:
#Merge data to compare
ensemble = pd.DataFrame([yhat_xbg_x, yhat_gb_x, yhat_rf_x]).T
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']
moda_x = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda_x

In [ ]:
#Prepare data to export
yhat_final = ensemble['pred']
df2_x['Reserva Cancelada'] = yhat_final
submission = df2_x[['id', 'Reserva Cancelada']].copy()

In [ ]:
#Export data
submission.to_csv('submission.csv', index=False)